In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 10 19:00:31 2019

@author: Neil Rodrigues
"""

cifar10_dataset_folder_path = 'cifar-10-batches-py'

import pickle
import numpy as np
import matplotlib.pyplot as plt  
import tensorflow as tf



def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')
       
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

def one_hot_encode(x):

    encoded = np.zeros((len(x), 10))
    
    for idx, val in enumerate(x):
        encoded[idx][val] = 1
    
    return encoded


def _preprocess_and_save(features, labels, filename):
    features = normalize(features)
    labels = one_hot_encode(labels)

    pickle.dump((features, labels), open(filename, 'wb'))

def _save(features, labels, filename):
    features = normalize(features)
    
    pickle.dump((features, labels), open(filename, 'wb'))
    
def preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode):
    n_batches = 5


    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_i)

        _preprocess_and_save(features, labels, 'preprocess_batch_' + str(batch_i) + '.p')

    # load the test dataset
    with open(cifar10_dataset_folder_path + '/test_batch', mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    # preprocess the testing data
    test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    test_labels = batch['labels']

    # Preprocess and Save all testing data
    _save(np.array(test_features), np.array(test_labels),'preprocess_training.p')



preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)



tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')


def conv_net(x):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 6], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 6, 12], mean=0, stddev=0.08))
    conv3_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 12, 24], mean=0, stddev=0.08))
   

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
  
    # 5, 6
    conv3 = tf.nn.conv2d(conv2_bn, conv3_filter, strides=[1,1,1,1], padding='SAME')
    conv3 = tf.nn.relu(conv3)
    conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  
    conv3_bn = tf.layers.batch_normalization(conv3_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv3_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=120, activation_fn=tf.nn.relu)

    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=84, activation_fn=tf.nn.relu)

    full2 = tf.layers.batch_normalization(full2)
    
    # 12
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
  
    return out




epochs = 20
batch_size = 128

learning_rate = 0.001

logits = conv_net(x)
model = tf.identity(logits, name='logits') 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

values,indices=tf.math.top_k(logits,k=1,name=None)




def train_neural_network(session, optimizer, feature_batch, label_batch):
    session.run(optimizer, 
                feed_dict={x: feature_batch,y: label_batch,})

def test_data(session, logits, feature_batch, label_batch):
    #index=session.run(indices,feed_dict={x:feature_batch})
    #print(label_batch.reshape((1,10000)).shape,index.reshape((1,10000)).shape)
    cm = tf.confusion_matrix(labels=tf.argmax(logits, 1), predictions=label_batch)
    c_1=sess.run(cm,feed_dict={x:feature_batch})
    den=np.sum(c_1)
    num=np.trace(c_1)
    accuracy=num/den
    print(accuracy)

    
def print_stats(session, feature_batch, label_batch, cost):
    loss = sess.run(cost, feed_dict={x: feature_batch,y: label_batch,})
    print('Loss: {:>10.4f} '.format(loss))
    
def batch_features_labels(features, labels, batch_size):
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

def load_preprocess_training_batch(batch_id, batch_size):
    filename = 'preprocess_batch_' + str(batch_id) + '.p'
    features, labels = pickle.load(open(filename, mode='rb'))
    return batch_features_labels(features, labels, batch_size)

def load_testing_batch():
    filename = 'preprocess_training.p'
    features, labels = pickle.load(open(filename, mode='rb'))
    return (features, labels)


save_model_path = './image_classification'

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = 5;
        for batch_i in range(1, n_batches + 1):
            for batch_features, batch_labels in load_preprocess_training_batch(batch_i, batch_size):
                train_neural_network(sess, optimizer, batch_features, batch_labels)
                
            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_features, batch_labels, cost)
    feature_batch,label_batch = load_testing_batch()
    test_data(sess, logits, feature_batch, label_batch)    
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Training...
Epoch  1, CIFAR-10 Batch 1:  Loss:     1.7889 
Epoch  1, CIFAR-10 Batch 2:  Loss:     1.4957 
Epoch  1, CIFAR-10 Batch 3:  Loss:     1.0468 
Epoch  1, CIFAR-10 Batch 4:  Loss:     1.3527 
Epoch  1, CIFAR-10 Batch 5:  Loss:     1.8071 
Epoch  2, CIFAR-10 Batch 1:  Loss:     1.3340 
Epoch  2, CIFAR-10 Batch 2:  Loss:     1.3132 
Epoch  2, CIFAR-10 Batch 3:  Loss:     0.8250 
Epoch  2, CIFAR-10 Batch 4:  Loss:     0.9254 
Epoch  2, CIFAR-10 Batch 5:  Loss:     1.4921 
Epoch  3, CIFAR-10 Batch 1:  Loss:     1.2166 
Epoch  3, CIFAR-10 Batch 2:  Loss:     0.9693 
Epoch  3, CIFAR-10 Batch 3:  Loss:     0.6856 
Epoch  3, CIFAR-10 Batch 4:  Loss:     0.6264 
Epoch  3, CIFAR-10 Batch 5:  Loss:     1.2731 
Epoch  4, CIFAR-10 Batch 1:  Loss:     0.9698 
Epoch  4, CIFAR-10 Batch 2:  Loss:     0.7196 
Epoch  4, CIFAR-10 Batch 3:  Loss:     0.5311 
Epoch  4, CIFAR-10 Batch 4:  Loss:     0.4811 
Epoch  4, CIFAR-10 Batch 5:  Loss:     1.0640 
Epoch  5, CIFAR-10 Batch 1:  Loss:     0.7512 
E